In [21]:
import findspark
import numpy
import sklearn
findspark.init()

from pyspark.sql.functions import *
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [6]:
ratings = spark.read.csv("ratings.csv",header=True)
movies = spark.read.csv("movies.csv",header=True)

In [10]:
ratings = ratings.\
    withColumn('userId',col('userId').cast('integer')).\
    withColumn('movieId', col('movieId').cast('integer')).\
    withColumn('rating',col('rating').cast('float')).\
    drop('timestamp')
ratings.show(5)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
+------+-------+------+
only showing top 5 rows



In [13]:
user_ratings = ratings.join(movies, on='movieId')
user_ratings = user_ratings.toPandas()
user_ratings = user_ratings.pivot_table(index=['userId'], columns=['title'], values='rating')
user_ratings.head()

title,"""11'09""""01 - September 11 (2002)""",'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
user_distances_spearman = user_ratings.transpose().corr(method='spearman')
user_distances_spearman

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,NaN,0.000000,0.322138,0.210156,-0.311114,-0.091952,0.445076,0.968246,-0.261116,...,0.083571,0.077652,-0.037408,-0.426028,-0.075944,0.101755,0.159039,0.255729,-0.155265,-0.038335
2,NaN,1.000000,NaN,NaN,NaN,NaN,-0.866025,NaN,NaN,0.100000,...,-0.399403,NaN,-1.000000,NaN,NaN,0.645497,NaN,0.000000,NaN,0.587499
3,0.000000,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.430498,NaN,NaN,-0.679366,-0.333333,-0.529412,NaN,0.356873
4,0.322138,NaN,NaN,1.000000,-0.419229,0.114717,0.511816,0.221069,NaN,0.444484,...,-0.239048,0.380117,0.082002,-0.086831,0.305182,0.235615,0.073690,-0.193123,0.000000,-0.126470
5,0.210156,NaN,NaN,-0.419229,1.000000,0.079093,0.253524,0.002003,NaN,-0.500000,...,0.000000,0.245788,0.249095,-0.016959,-0.373085,0.174781,0.242849,-0.006526,0.340168,0.068806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.101755,0.645497,-0.679366,0.235615,0.174781,-0.036029,0.160342,0.209321,0.467983,-0.216409,...,0.290239,0.088513,0.264487,0.631583,0.151663,1.000000,0.063323,0.265448,0.537500,0.333528
607,0.159039,NaN,-0.333333,0.073690,0.242849,0.248291,0.389227,0.227571,NaN,-0.666667,...,0.658090,0.207247,0.155523,-0.028324,-0.340897,0.063323,1.000000,0.194180,0.207870,0.074757
608,0.255729,0.000000,-0.529412,-0.193123,-0.006526,0.099418,-0.034199,0.429670,0.312278,-0.486046,...,0.479259,0.200657,0.107904,0.031309,-0.158151,0.265448,0.194180,1.000000,0.510460,0.117893


In [42]:
kneighbors = 5

user_distances_spearman = user_distances_spearman.dropna(axis=0, thresh=kneighbors+1).dropna(axis=1,
                                                                        thresh=kneighbors+1)

user_distances_spearman = user_distances_spearman.loc[user_distances_spearman.columns]

In [43]:
user_distances_spearman = 1.0 - user_distances_spearman

user_distances_spearman[user_distances_spearman < 0] = 0

In [44]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

model_knn = NearestNeighbors(metric='precomputed',algorithm='brute',n_neighbors=kneighbors, n_jobs=-1)

model_knn.fit(csr_matrix(user_distances_spearman.fillna(0).values))

similarity, indexes = model_knn.kneighbors(csr_matrix(user_distances_spearman.fillna(0).values),
                                         n_neighbors=kneighbors)

c:\programdata\anaconda3\envs\sparky\lib\site-packages\sklearn\neighbors\_base.py:199: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(


In [48]:
import pandas as pd

neighborhoods_spearman = pd.DataFrame({'neighborhood_ids':[user_distances_spearman.iloc[neighbors].index.to_list()
for neighbors in indexes],'distance': similarity.tolist()}, index=user_distances_spearman.index)

In [49]:
user_1_neighbors_spearman = neighborhoods_spearman['neighborhood_ids'].loc[1]
user_1_neighbors_spearman

[511, 9, 366, 154, 481]

In [50]:
user_ratings.loc[user_1_neighbors_spearman].mean().sort_values(ascending=False)[:10]

title
Looper (2012)                                 5.0
Reservoir Dogs (1992)                         5.0
Shawshank Redemption, The (1994)              5.0
Schindler's List (1993)                       5.0
E.T. the Extra-Terrestrial (1982)             5.0
Captain America: The Winter Soldier (2014)    5.0
Citizen Kane (1941)                           5.0
Rise of the Guardians (2012)                  5.0
Cloudy with a Chance of Meatballs (2009)      5.0
Good Will Hunting (1997)                      5.0
dtype: float32